In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm

REBULID_DATA = False

In [3]:
class DogsVSCats():
    imgsize = 48
    CATS = "PetImages/Cat"
    DOGS = "PetImages/Dog"   #directory names for data
    
    LABELS = {CATS: 0, DOGS: 1}  #labels for the data
    
    training_data = []
    catcount = 0
    dogcount = 0
    
    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)): #f is a file name in the label(directory)
                try: #weird data 가 들어올 수 있음
                    path = os.path.join(label, f)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (self.imgsize, self.imgsize))
                    self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])

                    if label == self.CATS:
                        self.catcount += 1
                    elif label == self.DOGS:
                        self.dogcount += 1
                except Exception as e:
                    pass
                        #print(str(excetion))
        np.random.shuffle(self.training_data)
        np.save("training_date.npy", self.training_data)
        print("Cats:", self.catcount)
        print("Dogs:", self.dogcount)
if REBULID_DATA:
    dogsvcats = DogsVSCats()
    dogsvcats.make_training_data()

  0%|                                                                              | 18/12501 [00:00<01:09, 178.68it/s]

PetImages/Cat


  0%|▏                                                                             | 29/12501 [00:00<00:43, 287.90it/s]

PetImages/Dog


100%|███████████████████████████████████████████████████████████████████████████| 12501/12501 [00:43<00:00, 287.42it/s]


Cats: 12476
Dogs: 12470


In [4]:
training_data = np.load("training_date.npy", allow_pickle=True)

In [5]:
print(len(training_data))

24946


In [6]:
print(training_data[1])

[array([[109, 107, 106, ..., 104,  96,  97],
       [108, 106, 108, ..., 104, 101,  97],
       [110, 108, 110, ..., 102, 100, 100],
       ...,
       [124, 128, 130, ..., 119, 112, 113],
       [126, 125, 130, ..., 111, 108, 113],
       [121, 131, 128, ..., 107, 111, 111]], dtype=uint8)
 array([0., 1.])]


In [7]:
import matplotlib.pyplot as plt

plt.imshow(training_data[1][0], cmap="gray")
plt.show()

<Figure size 640x480 with 1 Axes>

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            #24, 24, 32
            
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            #12, 12, 32
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            #6, 6, 64
            
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            #3, 3, 64
        )
        self.fcc = nn.Sequential(
            nn.Linear(576, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )
    def forward(self, x):
        x = self.cnn(x)
        x = x.view(x.size(0), -1)
        x = self.fcc(x)
        x = F.softmax(x, dim=1)
        return x

net = Net()

In [9]:
import torch.optim as optim
optimizer = optim.Adam(net.parameters(), lr=1e-3)
loss_function  = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1, 48, 48)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1
val_size = int(len(X)*VAL_PCT)


In [10]:
train_x = X[:-val_size]
train_y = y[:-val_size]

test_x = X[-val_size:]
test_y = y[-val_size:]

print(len(train_x))
print(len(test_x))

22452
2494


In [11]:
batch_size = 100
EPOCHS = 1

for epochs in range(EPOCHS):
    for i in tqdm(range(0, len(train_x), batch_size)):
        batch_x = train_x[i:i+batch_size].view(-1, 1,48,48)
        batch_y = train_y[i:i+batch_size]
        
        net.zero_grad()
        outputs = net(batch_x)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        if i%10 == 0:
            print(i, " th batch loss : ", loss)
        
        
        

  0%|▎                                                                                 | 1/225 [00:00<02:11,  1.70it/s]

0  th batch loss :  tensor(0.2412, grad_fn=<MseLossBackward>)


  1%|▋                                                                                 | 2/225 [00:01<02:07,  1.75it/s]

100  th batch loss :  tensor(0.2541, grad_fn=<MseLossBackward>)


  1%|█                                                                                 | 3/225 [00:01<01:59,  1.86it/s]

...

22200  th batch loss :  tensor(0.2231, grad_fn=<MseLossBackward>)


100%|███████████████████████████████████████████████████████████████████████████████▋| 224/225 [01:48<00:00,  2.02it/s]

22300  th batch loss :  tensor(0.2225, grad_fn=<MseLossBackward>)


100%|████████████████████████████████████████████████████████████████████████████████| 225/225 [01:48<00:00,  2.06it/s]

22400  th batch loss :  tensor(0.2438, grad_fn=<MseLossBackward>)


In [12]:
correct = 0
total = 0
with torch.no_grad():
    for i in range(len(test_x)):
        real_class = torch.argmax(test_y[i])
        prediction = net(test_x[i].view(-1, 1, 48, 48))
        print(prediction)
        predicted_class = torch.argmax(prediction)
        if predicted_class == real_class:
            correct += 1
        total += 1
print("accurcy : ", round(correct/total, 3))

tensor([[0.4215, 0.5785]])
tensor([[0.5181, 0.4819]])
tensor([[0.2832, 0.7168]])
tensor([[0.5287, 0.4713]])
tensor([[0.7200, 0.2800]])
tensor([[0.3577, 0.6423]])
tensor([[0.4767, 0.5233]])
tensor([[0.4131, 0.5869]])
tensor([[0.4190, 0.5810]])
tensor([[0.5396, 0.4604]])
tensor([[0.3961, 0.6039]])
tensor([[0.2035, 0.7965]])
tensor([[0.3015, 0.6985]])
tensor([[0.2632, 0.7368]])
tensor([[0.2628, 0.7372]])
tensor([[0.2436, 0.7564]])
tensor([[0.2641, 0.7359]])
tensor([[0.3977, 0.6023]])
tensor([[0.3753, 0.6247]])
tensor([[0.3225, 0.6775]])
tensor([[0.5242, 0.4758]])
tensor([[0.3733, 0.6267]])
tensor([[0.4740, 0.5260]])
tensor([[0.2677, 0.7323]])
tensor([[0.5018, 0.4982]])
tensor([[0.2807, 0.7193]])
tensor([[0.4403, 0.5597]])
tensor([[0.5876, 0.4124]])
tensor([[0.4519, 0.5481]])
tensor([[0.2986, 0.7014]])
tensor([[0.3067, 0.6933]])
tensor([[0.2913, 0.7087]])
tensor([[0.6429, 0.3571]])
tensor([[0.5266, 0.4734]])
tensor([[0.3811, 0.6189]])
tensor([[0.3075, 0.6925]])
tensor([[0.6548, 0.3452]])
t

tensor([[0.2594, 0.7406]])
tensor([[0.3945, 0.6055]])
tensor([[0.5418, 0.4582]])
tensor([[0.4000, 0.6000]])
tensor([[0.3505, 0.6495]])
tensor([[0.2783, 0.7217]])
tensor([[0.2571, 0.7429]])
tensor([[0.3915, 0.6085]])
tensor([[0.3878, 0.6122]])
tensor([[0.2718, 0.7282]])
tensor([[0.5330, 0.4670]])
tensor([[0.5618, 0.4382]])
tensor([[0.2321, 0.7679]])
tensor([[0.2863, 0.7137]])
tensor([[0.5965, 0.4035]])
tensor([[0.3175, 0.6825]])
tensor([[0.3733, 0.6267]])
tensor([[0.4549, 0.5451]])
tensor([[0.3369, 0.6631]])
tensor([[0.2568, 0.7432]])
tensor([[0.2052, 0.7948]])
tensor([[0.5669, 0.4331]])
tensor([[0.2767, 0.7233]])
tensor([[0.5076, 0.4924]])
tensor([[0.2608, 0.7392]])
tensor([[0.6404, 0.3596]])
tensor([[0.3702, 0.6298]])
tensor([[0.3196, 0.6804]])
tensor([[0.7567, 0.2433]])
tensor([[0.3350, 0.6650]])
tensor([[0.3107, 0.6893]])
tensor([[0.5336, 0.4664]])
tensor([[0.3898, 0.6102]])
tensor([[0.5054, 0.4946]])
tensor([[0.2185, 0.7815]])
tensor([[0.2811, 0.7189]])
tensor([[0.3945, 0.6055]])
t

tensor([[0.4884, 0.5116]])
tensor([[0.8013, 0.1987]])
tensor([[0.2363, 0.7637]])
tensor([[0.4046, 0.5954]])
tensor([[0.3521, 0.6479]])
tensor([[0.7006, 0.2994]])
tensor([[0.4670, 0.5330]])
tensor([[0.3722, 0.6278]])
tensor([[0.2596, 0.7404]])
tensor([[0.4499, 0.5501]])
tensor([[0.3493, 0.6507]])
tensor([[0.5687, 0.4313]])
tensor([[0.3068, 0.6932]])
tensor([[0.2544, 0.7456]])
tensor([[0.4507, 0.5493]])
tensor([[0.6093, 0.3907]])
tensor([[0.5710, 0.4290]])
tensor([[0.5124, 0.4876]])
tensor([[0.5197, 0.4803]])
tensor([[0.5774, 0.4226]])
tensor([[0.6751, 0.3249]])
tensor([[0.2648, 0.7352]])
tensor([[0.4236, 0.5764]])
tensor([[0.5510, 0.4490]])
tensor([[0.5391, 0.4609]])
tensor([[0.2614, 0.7386]])
tensor([[0.6700, 0.3300]])
tensor([[0.3707, 0.6293]])
tensor([[0.5872, 0.4128]])
tensor([[0.4258, 0.5742]])
tensor([[0.2667, 0.7333]])
tensor([[0.6927, 0.3073]])
tensor([[0.3910, 0.6090]])
tensor([[0.3276, 0.6724]])
tensor([[0.4624, 0.5376]])
tensor([[0.4106, 0.5894]])
tensor([[0.4558, 0.5442]])
t

tensor([[0.4085, 0.5915]])
tensor([[0.3026, 0.6974]])
tensor([[0.2608, 0.7392]])
tensor([[0.2651, 0.7349]])
tensor([[0.3646, 0.6354]])
tensor([[0.4114, 0.5886]])
tensor([[0.6435, 0.3565]])
tensor([[0.4293, 0.5707]])
tensor([[0.2396, 0.7604]])
tensor([[0.4025, 0.5975]])
tensor([[0.2819, 0.7181]])
tensor([[0.2981, 0.7019]])
tensor([[0.2047, 0.7953]])
tensor([[0.4446, 0.5554]])
tensor([[0.5993, 0.4007]])
tensor([[0.3437, 0.6563]])
tensor([[0.3956, 0.6044]])
tensor([[0.2329, 0.7671]])
tensor([[0.3895, 0.6105]])
tensor([[0.4853, 0.5147]])
tensor([[0.6773, 0.3227]])
tensor([[0.4963, 0.5037]])
tensor([[0.3066, 0.6934]])
tensor([[0.4568, 0.5432]])
tensor([[0.6490, 0.3510]])
tensor([[0.3027, 0.6973]])
tensor([[0.2468, 0.7532]])
tensor([[0.2921, 0.7079]])
tensor([[0.6550, 0.3450]])
tensor([[0.2810, 0.7190]])
tensor([[0.3535, 0.6465]])
tensor([[0.3578, 0.6422]])
tensor([[0.3566, 0.6434]])
tensor([[0.4346, 0.5654]])
tensor([[0.2619, 0.7381]])
tensor([[0.2598, 0.7402]])
tensor([[0.3700, 0.6300]])
t

tensor([[0.3250, 0.6750]])
tensor([[0.2740, 0.7260]])
tensor([[0.3937, 0.6063]])
tensor([[0.4543, 0.5457]])
tensor([[0.4539, 0.5461]])
tensor([[0.6244, 0.3756]])
tensor([[0.5343, 0.4657]])
tensor([[0.5320, 0.4680]])
tensor([[0.4982, 0.5018]])
tensor([[0.1922, 0.8078]])
tensor([[0.3785, 0.6215]])
tensor([[0.3101, 0.6899]])
tensor([[0.2541, 0.7459]])
tensor([[0.3779, 0.6221]])
tensor([[0.2350, 0.7650]])
tensor([[0.2968, 0.7032]])
tensor([[0.4118, 0.5882]])
tensor([[0.2016, 0.7984]])
tensor([[0.6680, 0.3320]])
tensor([[0.3429, 0.6571]])
tensor([[0.3171, 0.6829]])
tensor([[0.2457, 0.7543]])
tensor([[0.4965, 0.5035]])
tensor([[0.4496, 0.5504]])
tensor([[0.7075, 0.2925]])
tensor([[0.3112, 0.6888]])
tensor([[0.2950, 0.7050]])
tensor([[0.3549, 0.6451]])
tensor([[0.4243, 0.5757]])
tensor([[0.1852, 0.8148]])
tensor([[0.2185, 0.7815]])
tensor([[0.2496, 0.7504]])
tensor([[0.1827, 0.8173]])
tensor([[0.6646, 0.3354]])
tensor([[0.5685, 0.4315]])
tensor([[0.3297, 0.6703]])
tensor([[0.3855, 0.6145]])
t

tensor([[0.5763, 0.4237]])
tensor([[0.2674, 0.7326]])
tensor([[0.2262, 0.7738]])
tensor([[0.4911, 0.5089]])
tensor([[0.5361, 0.4639]])
tensor([[0.3630, 0.6370]])
tensor([[0.2357, 0.7643]])
tensor([[0.6276, 0.3724]])
tensor([[0.3438, 0.6562]])
tensor([[0.2347, 0.7653]])
tensor([[0.2994, 0.7006]])
tensor([[0.3876, 0.6124]])
tensor([[0.4356, 0.5644]])
tensor([[0.4220, 0.5780]])
tensor([[0.7578, 0.2422]])
tensor([[0.4568, 0.5432]])
tensor([[0.5884, 0.4116]])
tensor([[0.6636, 0.3364]])
tensor([[0.3604, 0.6396]])
tensor([[0.3104, 0.6896]])
tensor([[0.3468, 0.6532]])
tensor([[0.4778, 0.5222]])
tensor([[0.4764, 0.5236]])
tensor([[0.3787, 0.6213]])
tensor([[0.6165, 0.3835]])
tensor([[0.4572, 0.5428]])
tensor([[0.5493, 0.4507]])
tensor([[0.3748, 0.6252]])
tensor([[0.4546, 0.5454]])
tensor([[0.3260, 0.6740]])
tensor([[0.5753, 0.4247]])
tensor([[0.3895, 0.6105]])
tensor([[0.3991, 0.6009]])
tensor([[0.5477, 0.4523]])
tensor([[0.4522, 0.5478]])
tensor([[0.3895, 0.6105]])
tensor([[0.4124, 0.5876]])
t

tensor([[0.5396, 0.4604]])
tensor([[0.3966, 0.6034]])
tensor([[0.2686, 0.7314]])
tensor([[0.4769, 0.5231]])
tensor([[0.4214, 0.5786]])
tensor([[0.3226, 0.6774]])
tensor([[0.4107, 0.5893]])
tensor([[0.2639, 0.7361]])
tensor([[0.4267, 0.5733]])
tensor([[0.2467, 0.7533]])
tensor([[0.3752, 0.6248]])
tensor([[0.2875, 0.7125]])
tensor([[0.7233, 0.2767]])
tensor([[0.5614, 0.4386]])
tensor([[0.2668, 0.7332]])
tensor([[0.3960, 0.6040]])
tensor([[0.3228, 0.6772]])
tensor([[0.2786, 0.7214]])
tensor([[0.2546, 0.7454]])
tensor([[0.4372, 0.5628]])
tensor([[0.3893, 0.6107]])
tensor([[0.5344, 0.4656]])
tensor([[0.6520, 0.3480]])
tensor([[0.2883, 0.7117]])
tensor([[0.2216, 0.7784]])
tensor([[0.4550, 0.5450]])
tensor([[0.4211, 0.5789]])
tensor([[0.3182, 0.6818]])
tensor([[0.4124, 0.5876]])
tensor([[0.3021, 0.6979]])
tensor([[0.5313, 0.4687]])
tensor([[0.1810, 0.8190]])
tensor([[0.3314, 0.6686]])
tensor([[0.4294, 0.5706]])
tensor([[0.3852, 0.6148]])
tensor([[0.6606, 0.3394]])
tensor([[0.5421, 0.4579]])
t

tensor([[0.3980, 0.6020]])
tensor([[0.3215, 0.6785]])
tensor([[0.4871, 0.5129]])
tensor([[0.2997, 0.7003]])
tensor([[0.4291, 0.5709]])
tensor([[0.4736, 0.5264]])
tensor([[0.3864, 0.6136]])
tensor([[0.4647, 0.5353]])
tensor([[0.2217, 0.7783]])
tensor([[0.4550, 0.5450]])
tensor([[0.5519, 0.4481]])
tensor([[0.5137, 0.4863]])
tensor([[0.4515, 0.5485]])
tensor([[0.2930, 0.7070]])
tensor([[0.3715, 0.6285]])
tensor([[0.2252, 0.7748]])
tensor([[0.4171, 0.5829]])
tensor([[0.3044, 0.6956]])
tensor([[0.6493, 0.3507]])
tensor([[0.4049, 0.5951]])
tensor([[0.3599, 0.6401]])
tensor([[0.3463, 0.6537]])
tensor([[0.4268, 0.5732]])
tensor([[0.5335, 0.4665]])
tensor([[0.3480, 0.6520]])
tensor([[0.3316, 0.6684]])
tensor([[0.2346, 0.7654]])
tensor([[0.4181, 0.5819]])
tensor([[0.6945, 0.3055]])
tensor([[0.6135, 0.3865]])
tensor([[0.2724, 0.7276]])
tensor([[0.3635, 0.6365]])
tensor([[0.2831, 0.7169]])
tensor([[0.4037, 0.5963]])
tensor([[0.4763, 0.5237]])
tensor([[0.2968, 0.7032]])
tensor([[0.5208, 0.4792]])
t

tensor([[0.4966, 0.5034]])
tensor([[0.3908, 0.6092]])
tensor([[0.5978, 0.4022]])
tensor([[0.6547, 0.3453]])
tensor([[0.4371, 0.5629]])
tensor([[0.5026, 0.4974]])
tensor([[0.1881, 0.8119]])
tensor([[0.2109, 0.7891]])
tensor([[0.2823, 0.7177]])
tensor([[0.5587, 0.4413]])
tensor([[0.2858, 0.7142]])
tensor([[0.2953, 0.7047]])
tensor([[0.2631, 0.7369]])
tensor([[0.6266, 0.3734]])
tensor([[0.3746, 0.6254]])
tensor([[0.3043, 0.6957]])
tensor([[0.4169, 0.5831]])
tensor([[0.5662, 0.4338]])
tensor([[0.2985, 0.7015]])
accurcy :  0.648


In [14]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cpu
